In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report, 
    confusion_matrix, roc_auc_score, recall_score, precision_score
)
from sklearn.preprocessing import StandardScaler

# SMOTE for oversampling
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Models
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

# Utilities
import joblib
from datetime import datetime
import os

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("All libraries imported successfully!")

In [ ]:
# Load data
train_df = pd.read_csv('data/data_minihackathon_train_engineered.csv')
test_df = pd.read_csv('data/data_minihackathon_test_engineered.csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# Create binary labels
X = train_df.drop(['drug_category', 'id'], axis=1, errors='ignore')
y_original = train_df['drug_category']
X_test = test_df.drop(['ID', 'id'], axis=1, errors='ignore')
test_ids = range(501, 501 + len(test_df))

y_binary = (y_original == 'Depressants').astype(int)

print(f"\nClass distribution:")
print(f"  Depressants: {y_binary.sum()} ({y_binary.sum()/len(y_binary)*100:.2f}%)")
print(f"  Others: {(y_binary == 0).sum()} ({(y_binary == 0).sum()/len(y_binary)*100:.2f}%)")
print(f"  Imbalance ratio: 1:{(y_binary == 0).sum() / y_binary.sum():.2f}")

## Strategy 1: XGBoost with Extreme Class Weighting

In [ ]:
%%time

print("\n" + "="*80)
print("XGBoost with AGGRESSIVE Class Weighting (Prioritize Recall)")
print("="*80)

# Aggressive scale_pos_weight to force model to catch Depressants
scale_pos_weight = (y_binary == 0).sum() / y_binary.sum() * 1.5  # 1.5x boost

xgb_model = xgb.XGBClassifier(
    n_estimators=1000,
    max_depth=8,
    learning_rate=0.03,
    min_child_weight=1,  # Lower to allow smaller leaf nodes
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,  # No regularization on splits
    reg_alpha=0.1,  # Light L1
    reg_lambda=0.5,  # Light L2
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    tree_method='hist',
    device='cpu',
    eval_metric='logloss'
)

print(f"Scale pos weight: {scale_pos_weight:.2f}")

# Train and predict
xgb_model.fit(X, y_binary)
xgb_proba = xgb_model.predict_proba(X_test)[:, 1]

print(f"\n✓ XGBoost trained")
print(f"Probability range: {xgb_proba.min():.4f} - {xgb_proba.max():.4f}")

## Strategy 2: LightGBM with Balanced Training

In [ ]:
%%time

print("\n" + "="*80)
print("LightGBM with Balanced Class Weighting")
print("="*80)

lgb_model = lgb.LGBMClassifier(
    n_estimators=1000,
    num_leaves=50,
    learning_rate=0.03,
    min_data_in_leaf=10,  # Lower to allow smaller groups
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    class_weight='balanced',  # Auto-balance
    random_state=42,
    verbose=-1,
    force_col_wise=True
)

lgb_model.fit(X, y_binary)
lgb_proba = lgb_model.predict_proba(X_test)[:, 1]

print(f"✓ LightGBM trained")
print(f"Probability range: {lgb_proba.min():.4f} - {lgb_proba.max():.4f}")

## Strategy 3: Random Forest with SMOTE

In [ ]:
%%time

print("\n" + "="*80)
print("Random Forest with SMOTE Oversampling")
print("="*80)

# Apply SMOTE to create synthetic Depressants samples
smote = SMOTE(random_state=42, k_neighbors=5)
X_resampled, y_resampled = smote.fit_resample(X, y_binary)

print(f"Original: {y_binary.sum()} Depressants, {(y_binary==0).sum()} Others")
print(f"After SMOTE: {y_resampled.sum()} Depressants, {(y_resampled==0).sum()} Others")

rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_resampled, y_resampled)
rf_proba = rf_model.predict_proba(X_test)[:, 1]

print(f"✓ Random Forest trained on balanced data")
print(f"Probability range: {rf_proba.min():.4f} - {rf_proba.max():.4f}")

## Ensemble: Average Probabilities

In [ ]:
print("\n" + "="*80)
print("ENSEMBLE: Averaging All Models")
print("="*80)

# Average probabilities from all models
ensemble_proba = (xgb_proba + lgb_proba + rf_proba) / 3

print(f"\nEnsemble probability statistics:")
print(f"  Min: {ensemble_proba.min():.4f}")
print(f"  Max: {ensemble_proba.max():.4f}")
print(f"  Mean: {ensemble_proba.mean():.4f}")
print(f"  Median: {np.median(ensemble_proba):.4f}")

# Create results dataframe
results_df = pd.DataFrame({
    'id': list(test_ids),
    'xgb_proba': xgb_proba,
    'lgb_proba': lgb_proba,
    'rf_proba': rf_proba,
    'ensemble_proba': ensemble_proba
})

# Sort by ensemble probability
results_df = results_df.sort_values('ensemble_proba', ascending=False).reset_index(drop=True)

print(f"\nTop 20 most likely Depressants:")
print(results_df.head(20)[['id', 'ensemble_proba']].to_string(index=False))

## Select Top 63 as Depressants

In [ ]:
print("\n" + "="*80)
print("FINAL PREDICTIONS: Top 63 Samples as Depressants")
print("="*80)

# Select top 63 (confirmed ground truth count)
n_depressants = 63
top_63_ids = results_df.head(n_depressants)['id'].tolist()

# Create predictions
results_df['prediction'] = results_df['id'].apply(
    lambda x: 'Depressants' if x in top_63_ids else 'Others'
)

threshold = results_df.iloc[n_depressants-1]['ensemble_proba']
print(f"\nThreshold used: {threshold:.4f}")
print(f"Depressants predicted: {(results_df['prediction'] == 'Depressants').sum()}")

# Check your confirmed IDs
confirmed_ids = [513, 521, 570, 642, 770]
print(f"\n{'='*80}")
print(f"VALIDATION ON YOUR CONFIRMED DEPRESSANTS:")
print(f"{'='*80}")

for conf_id in confirmed_ids:
    row = results_df[results_df['id'] == conf_id].iloc[0]
    rank = results_df[results_df['id'] == conf_id].index[0] + 1
    status = "✓ CAUGHT" if row['prediction'] == 'Depressants' else "❌ MISSED"
    print(f"\nID {conf_id}:")
    print(f"  Ensemble prob: {row['ensemble_proba']:.4f}")
    print(f"  Rank: {rank}/{len(results_df)}")
    print(f"  Prediction: {row['prediction']} {status}")

caught = sum([1 for cid in confirmed_ids if cid in top_63_ids])
print(f"\n{'='*80}")
print(f"Caught {caught}/{len(confirmed_ids)} of your confirmed Depressants ({caught/len(confirmed_ids)*100:.1f}%)")
print(f"{'='*80}")

## Create Submission

In [ ]:
# Create submission
submission = results_df[['id', 'prediction']].copy()
submission.columns = ['id', 'drug_category']
submission = submission.sort_values('id').reset_index(drop=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f"submission_IMPROVED_ensemble_{timestamp}.csv"
submission.to_csv(filename, index=False)

print(f"\n{'='*80}")
print(f"✅ SUBMISSION CREATED: {filename}")
print(f"{'='*80}")
print(f"\nPrediction distribution:")
print(submission['drug_category'].value_counts())
print(f"\nFirst 10 predictions:")
print(submission.head(10).to_string(index=False))

# Save detailed results
os.makedirs('binary_predictions', exist_ok=True)
results_df.to_csv(f'binary_predictions/improved_ensemble_detailed_{timestamp}.csv', index=False)
print(f"\n✅ Detailed results: binary_predictions/improved_ensemble_detailed_{timestamp}.csv")

## Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Ensemble probability distribution
axes[0, 0].hist(ensemble_proba, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(x=threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {threshold:.3f}')
axes[0, 0].set_xlabel('Ensemble Probability')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Ensemble Probability Distribution')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Model comparison
model_names = ['XGBoost', 'LightGBM', 'RF+SMOTE', 'Ensemble']
model_probas = [xgb_proba, lgb_proba, rf_proba, ensemble_proba]
means = [p.mean() for p in model_probas]
axes[0, 1].bar(model_names, means, color=['steelblue', 'coral', 'lightgreen', 'gold'], edgecolor='black')
axes[0, 1].set_ylabel('Mean Probability')
axes[0, 1].set_title('Average Depressant Probability by Model')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Plot 3: Top 80 probabilities
top_80 = results_df.head(80)
colors = ['red' if p == 'Depressants' else 'blue' for p in top_80['prediction']]
axes[1, 0].scatter(range(len(top_80)), top_80['ensemble_proba'], c=colors, alpha=0.6)
axes[1, 0].axhline(y=threshold, color='black', linestyle='--', linewidth=1)
axes[1, 0].set_xlabel('Rank')
axes[1, 0].set_ylabel('Ensemble Probability')
axes[1, 0].set_title('Top 80 Samples by Probability (Red=Depressants, Blue=Others)')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Prediction counts
pred_counts = submission['drug_category'].value_counts()
axes[1, 1].bar(pred_counts.index, pred_counts.values, color=['steelblue', 'coral'], edgecolor='black')
axes[1, 1].set_ylabel('Count')
axes[1, 1].set_title('Final Prediction Distribution')
axes[1, 1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(pred_counts.values):
    axes[1, 1].text(i, v + 5, str(v), ha='center', fontweight='bold')

plt.tight_layout()
os.makedirs('visualizations', exist_ok=True)
plt.savefig('visualizations/improved_ensemble_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualization saved: visualizations/improved_ensemble_analysis.png")